In [ ]:
from lmfit import Model, Parameters
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

In [ ]:
def make_param_0():
    params = Parameters()
    # n_pre = dict0['n']

    # for i in range(n_pre):
    #     params.add(f"K{i}", dict0[f'K{i}'], vary = False) #min=0.9*dict0[f'K{i}'], max=1.1*dict0[f'K{i}'])
    #     params.add(f"b{i}", dict0[f'b{i}'], vary = False) #min=0.9*dict0[f'b{i}'], max=1.1*dict0[f'b{i}'])
    #     params.add(f"c{i}", dict0[f'c{i}'], vary = False) #min=0.9*dict0[f'c{i}'], max=1.1*dict0[f'c{i}'])
    #     params.add(f"d{i}", dict0[f'd{i}'], vary = False) #min=0.9*dict0[f'c{i}'], max=1.1*dict0[f'c{i}'])

    # for i in np.arange(n_pre,n):
    #     params.add(f"K{i}", 10, min=0)
    #     params.add(f"b{i}", 100, min=dict0[f'b{i-1}'])
    #     params.add(f"c{i}", 0.01, min=0)
    #     params.add(f"d{i}", 0, min=0)
    params.add(f"K", 10, min=0, max=100)
    params.add(f"b", 1, min=1e-2, max=100)
    params.add(f"c", 1, min=1e-6, max=2)
    #params.add(f"d", 0, min=0)

    return params

# Define the Gompertz bell curve function
def gompertz(t, K, b, c):
    return K*b*c*np.exp(-c*(t))*np.exp(-b*np.exp(-c*(t)))

# Fit on the S curve
def cum_gompertz(t, K, b, c):
    return np.cumsum(gompertz(t, K, b, c)) # + (cum_pos[-1]-d-np.cumsum(richardson(t, K, r, tm, nu))[-1])/(t[-1]-t[0])*(t-t[0])

# Dayly correction
def err_by_day(data, fit):
    sub_data=data[-7:0:-7]
    sub_fit=fit[-7:0:-7]
    err=0.4*(sub_fit-sub_data)/sub_data
    return np.mean(err)

In [ ]:
national_data=pd.read_csv('..\data\dati-andamento-nazionale\dpc-covid19-ita-andamento-nazionale.csv')
new_pos_tot=national_data['nuovi_positivi'].values[:]
cum_pos_tot=np.cumsum(new_pos_tot)
T_tot=new_pos_tot.shape[0]

In [ ]:
t0=200
T_tot = 300
fig, ax = plt.subplots(1,3)
fig.set_figheight(4)
fig.set_figwidth(16)
ax[0].grid(True)
ax[1].grid(True)
ax[2].grid(True)
ax[0].set_ylim(0,1.2*np.amax(new_pos_tot[t0:T_tot]))
param_list=[]
for t in range(t0+14,T_tot):
    new_pos = new_pos_tot[t0:t]  # Data acquisition
    cum_pos = np.cumsum(new_pos)
    if t==t0+14:
        norm_factor = np.amax(new_pos)
    else:
        norm_factor = np.amax(new_pos)
    data = new_pos/norm_factor # Normalization
    cum_data = np.cumsum(data)
    d0 = cum_data[0]
    tt=np.arange(t0,t)
    tt_forecast=np.arange(t0,t+14)

    # Create model
    if t==t0+14:
        params = make_param_0()
    else:
        params=result.params
    fmodel= Model(gompertz)

    # fit the model
    result = fmodel.fit(data, params, t=tt-t0, max_nfev=10000)
    sigma = result.eval_uncertainty()
    if t==t0+14:
        forecast=new_pos[-1]
        err=0
    else:
        forecast = fit[t-t0]
        err = err_by_day(new_pos[:-1], fit[0:t-1-t0])
    fit = norm_factor*gompertz(tt_forecast-t0, result.params['K'].value, result.params['b'].value, result.params['c'].value)
    
    # Saving parameters
    param_list.append([result.params['K'].value, result.params['b'].value, result.params['c'].value])

    # Visualization
    ax[0].scatter(t,new_pos[-1], color='b', marker='.')
    ax[0].scatter(t,forecast, color='r', marker='.')
    #ax[0].scatter(t,forecast*(1-err), color='g', marker='.')
    ax[1].scatter(t,(new_pos[-1]-forecast)/new_pos[-1], color='orange', marker='.')
    
    # ax[1].scatter(t,abs(new_pos[-1]-forecast*(1-err))/new_pos[-1], color='magenta', marker='.')
    # ax[1].annotate("",
    #         xy=(t, abs(new_pos[-1]-forecast*(1-err))/new_pos[-1]), xycoords='data',
    #         xytext=(t, abs(new_pos[-1]-forecast)/new_pos[-1]), textcoords='data',
    #         arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))
    viridis = mpl.cm.get_cmap('viridis', T_tot-t0)
    ax[2].plot(tt_forecast, fit, color=viridis((t-t0)/(T_tot-t0)))
    
ax[1].axhline(0, color='k', alpha=0.5)
ax[2].plot(tt, new_pos, color='red', label='data')
ax[0].legend(['data','forecast'])
ax[1].legend(['relative error'])
ax[2].legend()
plt.show()




In [ ]:
plt.figure(figsize=(10,5))
plt.grid(True)
for t in range(t0+14,T_tot):
    new_pos = new_pos_tot[t0:t]  # Data acquisition
    cum_pos = np.cumsum(new_pos)
    if t==t0+14:
        norm_factor = np.amax(new_pos)
    else:
        norm_factor = np.amax(new_pos)
    data = new_pos/norm_factor # Normalization
    cum_data = np.cumsum(data)
    d0 = cum_data[0]
    tt=np.arange(t0,t)
    tt_forecast=np.arange(t0,t+14)

    # Create model
    if t==t0+14:
        params = make_param_0()
    else:
        params=result.params
    fmodel= Model(gompertz)

    # fit the model
    result = fmodel.fit(data, params, t=tt-t0, max_nfev=10000)
    sigma = result.eval_uncertainty()
    if t==t0+14:
        forecast=new_pos[-1]
        err=0
    else:
        forecast = fit[t-t0]
        err = err_by_day(new_pos[:-1], fit[0:t-1-t0])
    fit = norm_factor*gompertz(tt_forecast-t0, result.params['K'].value, result.params['b'].value, result.params['c'].value)
    
    # Saving parameters
    param_list.append([result.params['K'].value, result.params['b'].value, result.params['c'].value])

    viridis = mpl.cm.get_cmap('viridis', T_tot-t0)
    plt.plot(tt_forecast, fit, color=viridis((t-t0)/(T_tot-t0)))
plt.plot(tt, new_pos, color='red', label='data')
plt.legend()

In [ ]:
print(result.fit_report())

labels=['K','b','c']
colors=['r','b','g']
for i in range(3):
    plt.figure()
    for j in range(len(param_list)):
        plt.scatter(j,param_list[j][i], marker='.',color=colors[i])
    plt.grid(True)
    plt.legend(labels[i])
    plt.show()

In [ ]:
# Using the forecast as a function

from lmfit import Model, Parameters
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import Forecast as fc

national_data=pd.read_csv('..\data\dati-andamento-nazionale\dpc-covid19-ita-andamento-nazionale.csv')
new_pos_tot=national_data['nuovi_positivi'].values[:]
cum_pos_tot=np.cumsum(new_pos_tot)
T_tot=new_pos_tot.shape[0]

t0=200
T_tot = 300
param_list=[]
forecast_list=[]
corrected_forecast_list=[]
rt_list=[]
conc_list=[]
for t in range(t0+14,T_tot):
    new_pos = new_pos_tot[t0:t]  # Data acquisition
    new_pos_avg = fc.rolling_avg(new_pos)

    daily_err = fc.compute_derr(new_pos,new_pos_avg)
    # daily_err = np.load('../data/daily_errors.npy')[(t+1-3)%7]

    # Compute rt
    rt_list.append(fc.compute_rt(new_pos_avg))

    # Compute concavity
    conc_list.append(fc.copute_conc(new_pos_avg))

    # Correct correction
    if t>t0+24:
        daily_err += 75*conc_list[-1]
    
    # Define parameters
    if t==t0+14:
        params = fc.make_param_0()
    else:
        params=result_params
    
    fit, result_params, forecast = fc.make_forecast(new_pos_avg, params)
    
    # Saving parameters and forecast
    param_list.append([result_params['K'].value, result_params['b'].value, result_params['c'].value])
    forecast_list.append(forecast)
    corrected_forecast_list.append(forecast*(1+daily_err))

# Visualization
fig, ax = plt.subplots(3,1)
fig.set_figheight(10)
ax[0].grid(True)
ax[1].grid(True)
ax[2].grid(True)
ax[0].set_ylim(0,1.2*np.amax(new_pos_tot[t0:T_tot]))
for t in range(t0+15,T_tot):
    ax[0].scatter(t,new_pos_tot[t], color='b', marker='.')
    ax[0].scatter(t,forecast_list[t-t0-15], color='r', marker='.')
    ax[0].scatter(t,corrected_forecast_list[t-t0-15], color='g', marker='.')
    ax[1].scatter(t,(new_pos_tot[t]-forecast_list[t-t0-15])/new_pos_tot[t], color='orange', marker='.')
    ax[1].scatter(t,(new_pos_tot[t]-corrected_forecast_list[t-t0-15])/new_pos_tot[t], color='magenta', marker='.')
    ax[1].annotate("",
            xy=(t, (new_pos_tot[t]-corrected_forecast_list[t-t0-15])/new_pos_tot[t]), xycoords='data',
            xytext=(t, (new_pos_tot[t]-forecast_list[t-t0-15])/new_pos_tot[t]), textcoords='data',
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))
    ax[2].scatter(t,rt_list[t-t0-15], color='g', marker='.')
ax[0].plot(np.arange(t0-3.5,T_tot-1-3.5),new_pos_avg, '-m')
ax[1].plot(np.arange(t0+15,T_tot+1), 100*np.array(conc_list).reshape(-1), 'g-')
ax[1].axhline(0, color='k', alpha=0.5)
ax[0].legend(['data','forecast','corrected'])
ax[1].legend(['error','error after correction'])
ax[2].legend(['rt'])
plt.show()


        

In [ ]:
# Using the forecast as a class

from lmfit import Model, Parameters
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import Forecast as fc

national_data=pd.read_csv('..\data\dati-andamento-nazionale\dpc-covid19-ita-andamento-nazionale.csv')
new_pos_tot=national_data['nuovi_positivi'].values[:]
cum_pos_tot=np.cumsum(new_pos_tot)
t0=200
T_tot=300

WF = fc.wave_fit(new_pos_tot)
WF.fit(t0,T_tot)
forecast_list = WF.forecast_list
corrected_forecast_list = WF.corrected_forecast_list



# Visualization
fig, ax = plt.subplots(2,1)
fig.set_figheight(10)
ax[0].grid(True)
ax[1].grid(True)
ax[0].set_ylim(0,1.2*np.amax(new_pos_tot[t0:T_tot]))
for t in range(t0+15,T_tot):
    ax[0].scatter(t,new_pos_tot[t], color='b', marker='.')
    ax[0].scatter(t,forecast_list[t-t0-15], color='r', marker='.')
    ax[0].scatter(t,corrected_forecast_list[t-t0-15], color='g', marker='.')
    ax[1].scatter(t,(new_pos_tot[t]-forecast_list[t-t0-15])/new_pos_tot[t], color='orange', marker='.')
    ax[1].scatter(t,(new_pos_tot[t]-corrected_forecast_list[t-t0-15])/new_pos_tot[t], color='magenta', marker='.')
    ax[1].annotate("",
            xy=(t, (new_pos_tot[t]-corrected_forecast_list[t-t0-15])/new_pos_tot[t]), xycoords='data',
            xytext=(t, (new_pos_tot[t]-forecast_list[t-t0-15])/new_pos_tot[t]), textcoords='data',
            arrowprops=dict(arrowstyle="->", connectionstyle="arc3"))
ax[0].plot(np.arange(t0-3.5,T_tot-1-3.5),WF.new_pos_avg, '-m')
ax[1].axhline(0, color='k', alpha=0.5)
ax[0].legend(['data','forecast','corrected'])
ax[1].legend(['error','error after correction'])
plt.show()


        